In [1]:
from random import randint
import random
import csv
import datetime
import hashlib
import re

# Static values
class static:
    MACHINES = 33
    LOCATIONS = 8
    EMPLOYEES = 29
    USERS = 23
    TASKSIZE = 9250
    JOBS = 39
    TIMESHEETS = 2250
    START_DATE = datetime.date(2021, 6, 1)
    DATE_RANGE = 120
    PROD_LOW_END = 58
    PROD_HIGH_END = 105
    DUMMY_PRODUCTIVITY_HIGH = [95, 95, 95, 100,
                            100, 100, 100, 100, 100, 105]
    DUMMY_PRODUCTIVITY_AVG = [50, 50, 50, 52, 55, 57, 60, 65, 65,
                          70, 75, 75, 79, 80, 80, 80, 82, 89, 
                          92, 92, 92, 93, 93, 93, 96, 96, 96,
                          96, 98, 98, 98, 100, 100, 100, 98,
                          98, 98, 100, 100, 100, 100, 102,
                          102, 102, 105, 105, 105, 110, 110]
    DUMMY_PRODUCTIVITY_LOW = [30, 35, 35, 45, 45, 50, 50, 50,
                            60, 60, 65, 70, 70, 75, 75, 80, 90,
                            90, 95, 95, 100, 100, 100, 100, 100]

# Converts list of tuples into CSV file
# List in format [(1, 'Dummy Data', 'Test', 3), (2, 'Real Data', 'Hi Mum', 22)]
# Converted to:
# 1, 'Dummy Data', 'Test', 3
# 2, 'Real Data', 'Hi Mum', 22

def write2csv(file, data, header):
    f = open(file, 'w', newline='')
    obj = csv.writer(f)
    obj.writerow(header)
    obj.writerows(data)
    f.close()

# hash salt generator
def generate_salt():
    ALPHABET = "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
    chars = []
    for i in range(16):
        chars.append(random.choice(ALPHABET))
    return "".join(chars)


In [2]:
# OBJECT: MACHINE
# machine_id,desc,location_id

data = []
for x in range(1, static.MACHINES+1):
    data.append(
        tuple([
            x,
            'Machine '+str(x),
            randint(1, static.LOCATIONS)
        ])
    )
    
write2csv('machines.csv', data, ['machine_id', 'desc', 'location_id'])

In [3]:
# OBJECT: LOCATION
# location_id, desc

data = []
for x in range(1, static.LOCATIONS+1):
    data.append(
        tuple([
            x,
            'Room '+str(x)
        ])
    )

write2csv('locations.csv', data, ['location_id', 'desc'])

In [4]:
# OBJECT: USER_TYPE
# type_id, desc, level, status

data = [(1, "Welder", 1, 1), (2, "Supervisor", 2, 1), (3, "Director", 2, 1), (10, "Admin", 5, 1)]

write2csv('user_types.csv', data, ['type_id', 'desc', 'level', 'status'])

In [5]:
# OBJECT: PAYROLL_USER (SmartFab RDBMS)
# payroll_id, first_name, last_name, employment_status

data = []
first_names = ['Michael', 'Christopher', 'Amanda', 'Jason', 'Jessica', 'David', 'Melissa', 'James', 'Matthew', 'Joshua', 'John', 'Robert', 'Joseph', 'Michelle', 'Daniel', 'Scott', 'Susan', 'Keith', 'Mohammed', 'Maria', 'Nushi', 'Jose', 'Wei', 'Ahmed', 'Muhammad', 'Ali', 'Yan', 'Li', 'Ana', 'Ying', 'Juan', 'Jean', 'Carlos', 'Antonio', 'Hui', 'Elena']
last_names = ['Smith', 'Smith', 'Johnson', 'Williams', 'Brown', 'Brown', 'Jones', 'Garcia', 'Miller', 'Davis', 'Rodrigeuz', 'Rodrigeuz', 'Martinez', 'Hernadez', 'Lopez', 'Wilson', 'Anderson', 'Thomas', 'Taylor', 'Moore', 'Jackson', 'Martin', 'Lee', 'Thompson', 'Martin', 'Lee', 'Thompson', 'White', 'Harris', 'Clark', 'Lewis', 'Robinson', 'Walker', 'Allen', 'Hill', 'Nguyen', 'Adams', 'Morales', 'Peterson', 'Patel', 'Kim', 'Choi', 'Jeong', 'de Silva', 'Perera', 'Fernando', 'Kumara', 'Chan', 'Wong', 'Lei', 'Kaya',
              'Demir', 'Tran', 'Wang', 'Li', 'Zhang', 'Chen', 'Liu', 'Devi', 'Yang', 'Huang', 'Singh', 'Wu', 'Wu', 'Kumar', 'Xu', 'Zhao', 'Luo', 'Ray', 'Mitchell', 'Carrillo', 'Hayat', 'Oaron', 'Khalaf', 'Amir', 'Carter', 'Swain', 'Mallik', 'Carter', 'Swain', 'Mallik', 'de Carvalho', 'Parra', 'Stewart', 'Nasir', 'Verma', 'Kanwar', 'Nasir', 'Verma', 'Kanwar', 'Antonio', 'Nair', 'Celik', 'Pedro', 'Bell', 'Lucas', 'Fischer', 'Wood', 'Meza', 'Hansen', 'Jang', 'Tahir', 'Cook', 'Watson', 'Rogers', 'Ward', 'Roman', 'Richards', 'Manna', 'Bailey']

for x in range(1, static.EMPLOYEES):
    fn = first_names[randint(0, len(first_names)-1)]
    ln = last_names[randint(0, len(last_names)-1)]
    data.append(
        tuple([
            format(randint(1, 999999), '06'),
            fn,
            ln,
            1
        ])
    )

    last_names.remove(ln)

write2csv('payroll_users.csv', data, ['payroll_id', 'first_name', 'last_name', 'employment_status'])


In [6]:
# OBJECT: TIMESHEET (SmartFab RDBMS)
# timesheet_id, payroll_id, date, shift_start, shift_end

data, pids, dates = [], [], []
shifts = [('08:00', '16:00'), ('09:00', '13:00'), ('09:00', '17:00'), ('09:30', '16:30'), ('12:00', '17:00'), ('12:30', '18:00')]

# Generate payroll ids
with open('payroll_users.csv', newline='') as f:
    l = csv.reader(f, delimiter=',')
    for r in l:
        try:
            pids.append(int(r[0]))
        except:
            continue

# Generate dates
for x in range(1,static.TIMESHEETS):
    random_date = static.START_DATE + datetime.timedelta(days=random.randrange(static.DATE_RANGE))
    dates.append(random_date)
dates = sorted(dates)

emp_count = 0
for x in dates:
    try:
        employee = pids[emp_count]
    except:
        emp_count = 0
        employee = pids[emp_count]
    emp_count += 1
    
    shift = shifts[randint(0, len(shifts)-1)]
    random.seed(emp_count)
    tid = hashlib.md5((str(employee) + str(shift) + str(x) + str(randint(emp_count,int(employee)))).encode('utf-8')).hexdigest()
    data.append(
        tuple([
            tid, #timesheet ID generated MD5 hash
            int(employee), #payroll_id
            x, # timesheet date
            shift[0], #start time
            shift[1], #end time
        ])
    )

write2csv('timesheet_data.csv', data, ['timesheet_id', 'payroll_id', 'date', 'shift_start', 'shift_end'])

In [7]:
#OBJECT: TIMESHEET_ENTRY
# tid, org_timesheet_id, payroll_id, date, shift_start, shift_end, shift_length

data = []
id_counter = 1
with open('timesheet_data.csv', newline='') as f:
    l = csv.reader(f, delimiter=',')
    for r in l:
        try:
            start_hr, start_min = int(re.split('[:]',r[3])[0]), int(re.split('[:]',r[3])[1])
            end_hr, end_min = int(re.split('[:]',r[4])[0]), int(re.split('[:]',r[4])[1])
            shift_len = ((datetime.timedelta(hours=end_hr, minutes=end_min) - datetime.timedelta(hours=start_hr, minutes=start_min)).total_seconds() / 3600)

            data.append(
                tuple([
                    id_counter, #tid
                    r[0], #original timesheet ID
                    r[1], #payroll ID
                    r[2], #timesheet date
                    r[3], #shift start
                    r[4], #shift end
                    shift_len, #shift length
                ])
            )

            id_counter += 1
        except:
            continue
        
write2csv('timesheets.csv', data, ['tid', 'org_timesheet_id', 'payroll_id', 'date', 'shift_start', 'shift_end', 'shift_length'])

In [8]:
#OBJECT: USER_LIST
# payroll_id, first_name, last_name, employment_status

data = []
with open('payroll_users.csv', newline='') as f:
    l = csv.reader(f, delimiter=',')
    for r in l:
        try:
            data.append(
                tuple([
                    int(r[0]), #payroll_id
                    r[1], #first_name
                    r[2], #last_name
                    int(r[3]), #employment_status
                ])
            )
        except:
            continue

write2csv('user_list.csv', data, ['payroll_id', 'first_name', 'last_name', 'employment_status'])

In [9]:
#OBJECT: USER
# uid, payroll_id, first_name, last_name, status, timesheet_status, user_types

data = [(1, 1000001, 'Lachlan', 'Langmead', 1, 0, [1, 2, 3, 10]), (2, 1000002, 'Mark', 'Lee', 1, 0, [1, 2, 3, 10])] #Add Developers
users = []
user_sample = random.sample(range(3,static.EMPLOYEES-1), static.USERS)
random_types = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3] # weight role to be more likely to select user type '1' (Welder)

# Read current user list into list
with open('user_list.csv', newline='') as f:
    l = csv.reader(f, delimiter=',')
    for r in l:
        try:
            users.append(
                tuple([
                    int(r[0]), #payroll_id
                    r[1], #first_name
                    r[2], #last_name
                    int(r[3]), #employment_status
                ])
            )
        except:
            continue

for x in user_sample:
    role = random_types[randint(0, len(random_types)-1)]
    data.append(
        tuple([
            x, #uid
            users[x][0], #payroll_id
            users[x][1], #first_name
            users[x][2], #last_name
            users[x][3], #employment_status
            1 if role == 1 or role == 2 else 0, #timesheet_status,
            [role]
        ])
    )

write2csv('users.csv', data, ['uid', 'payroll_id', 'first_name', 'last_name', 'status', 'timesheet_status', 'user_types'])

In [10]:
#OBJECT: CATEGORY
# category_id, arctime

data = [
    (1, 10),
    (2, 20),
    (3, 30),
    (4, 40),
    (5, 50)
]

write2csv('categories.csv', data, ['category_id', 'arctime'])


In [11]:
#OBJECT: JOB
# job_id, category, start_date, end_date, total_hrs
data = []
jids = random.sample(range(7000, 9000), static.JOBS)
jids = sorted(jids)

for x in range(1, len(jids)):
    random_date = static.START_DATE + datetime.timedelta(days=random.randrange(60))
    dates.append(random_date)
dates = sorted(dates)

categories = []
with open('categories.csv', newline='') as f:
    l = csv.reader(f, delimiter=',')
    next(f, None) # skip headers
    for r in l:
        categories.append(int(r[0]))

for x in jids:
    category = categories[randint(0, len(categories)-1)] # random category
    start_date = dates[randint(0, len(dates)-1)] # random start date

    # random end date based on category, smaller jobs = less time to take, with 70% chance of being finished
    end_date = (start_date + datetime.timedelta(days=random.randrange(category))) if randint(1,10) < 7 else None

    # random job arctime lengths
    job_lengths = [25, 25, 30, 50, 50, 50, 50, 75, 75, 85, 90, 90, 100, 100, 100, 100, 110, 125, 125, 150, 200, 200, 250, 275, 275, 300] 
    data.append(
        tuple([
            x, #job id
            category, # random category
            start_date, #start date
            0, #end date, all zero to begin with
            0,
            job_lengths[randint(0, len(job_lengths)-1)]
        ])
    )

write2csv('jobs.csv', data, ['job_id', 'category', 'start_date', 'end_date', 'accum_hrs', 'total_hrs'])

In [12]:
#OBJECT: TASK_TYPE
# task_type_id, desc

data = [
    (100, "login"),
    (101, "logout"),
    (404, "unknown error")
]

write2csv('task_types.csv', data, ['task_type_id', 'desc'])

In [13]:
def add2log(user_id, type_id, job_id, machine_id, dt):
    #task id generator
    tid = hashlib.md5((
        str(user_id) + # User ID
        str(dt) + # Clock in time
        str(datetime.datetime.now()) + # Current System Time
        str(machine_id) + # Machine ID of login machine
        generate_salt() # Random Salt
        ).encode('utf-8')).hexdigest()
    
    # task_id, uid, type, job_id, machine_id, datetime
    # IoT Login = 100
    return tuple([
            tid, #task id
            user_id, #user id
            type_id, #type = 100 = login
            job_id, #job id
            machine_id, #machine id
            dt.strftime("%Y-%m-%d %H:%M:%S")
        ])
        

def find_job(d):
    jids, data = [], []
    found = None
    date_dt = datetime.datetime.strptime(d, '%Y-%m-%d')
    with open('jobs.csv', newline='') as f:
        next(f, None)  # skip headers
        l = csv.reader(f, delimiter=',')
        for r in l:
            if (r[3] != '0'):
                continue

            start_date = datetime.datetime.strptime(r[2], '%Y-%m-%d')
            try:
                end_date = datetime.datetime.strptime(r[3], '%Y-%m-%d')
            except:
                end_date = datetime.datetime.today()

            if (date_dt >= start_date and date_dt <= end_date):
                jids.append(
                    tuple([
                        r[0],
                        r[1],
                        r[2],
                        r[3],
                        r[4]
                    ])
                )

    if len(jids) > 0:
        return jids[randint(0, len(jids)-1)]
    else:
        return [-1]


def inc_job_hrs(jid, hrs, date):
    f = open('jobs.csv')
    r = csv.reader(f)
    next(r, None)  # skip headers
    lines = list(r)
    count = 0

    while count < len(lines):
        if int(lines[count][0]) == jid:
            new_hrs = float(lines[count][4]) + hrs
            lines[count][4] = new_hrs
            if (float(lines[count][4]) >= float(lines[count][5])):
                if (randint(0, 10) > 8): # 80% chance of going over accum hours
                    lines[count][3] = str(date)[:10] # reset enddate to date of shift

        count += 1

    f.close()
    f = open('jobs.csv', 'w', newline='')
    w = csv.writer(f)
    w.writerow(['job_id', 'category', 'start_date',
                'end_date', 'accum_hrs', 'total_hrs'])
    w.writerows(lines)
    f.close()


#OBJECT: TASK
# task_id, uid, type, job_id, machine_id, datetime

data, uids, mids = [], [], []

# Generate task ids from random sample
temp_list = list(range(1, (static.TASKSIZE*2)))
task_id_list = [
    temp_list[i] for i in sorted(random.sample(range(len(temp_list)), static.TASKSIZE))
]

# Build user ids
with open('user_list.csv', newline='') as f:
    l = csv.reader(f, delimiter=',')
    for r in l:
        try:
            uids.append(int(r[0]))
        except:
            continue

user_ratings = []
for u in uids:
    user_ratings.append(tuple([
        u,
        randint(1, 10)
    ]))

# Build machine ids
with open('machines.csv', newline='') as f:
    l = csv.reader(f, delimiter=',')
    for r in l:
        try:
            mids.append(int(r[0]))
        except:
            continue

uids = uids[2:] # Remove developer IDs

master_mids = mids.copy()
current_day = None

with open('timesheets.csv', newline='') as f:
    next(f, None) # skip headers
    l = csv.reader(f, delimiter=',')
    new_mid_list = False
    for r in l:

        chance_of_shift = True if randint(0,100)<99 else False # Chance of shift is 99%, chances of calling in sick/no-shows/holidays/etc
        
        if (datetime.datetime.strptime(r[3], '%Y-%m-%d') != current_day):
            mids = master_mids.copy()

        current_day = datetime.datetime.strptime(r[3], '%Y-%m-%d')

        if (chance_of_shift):

            current_user = int(r[2])

            current_job_master = find_job(r[3])
            current_job = int(current_job_master[0])  # job id

            if (current_job == -1):
                continue

            current_job_arctime = (int(current_job_master[1])*10)

            try:
                current_machine = mids[randint(0, len(mids)-1)]
                mids.remove(current_machine)
            except:
                continue

            shift_length = float(r[6])
            start_hr, start_min = int(re.split('[:]',r[4])[0]), int(re.split('[:]',r[4])[1])
            end_hr, end_min = int(re.split('[:]',r[5])[0]), int(re.split('[:]',r[5])[1])
            date = [int(x) for x in r[3].split('-')]
            shift_year, shift_month, shift_day = date

            dt_start = datetime.datetime(shift_year, shift_month, shift_day, start_hr, start_min, 0)
            dt_end = datetime.datetime(shift_year, shift_month, shift_day, end_hr, end_min, 0)

            ############# PADDING AMOUNTS #################
            ### Factors in random lost time events/late arrival/early start/etc

            # Actual time expected worked = arc_shift
            # Adjusting shift = arc_padding
            # Shift_length (in hrs) x (expected arctime/60)
            
            arc_shift = shift_length * (current_job_arctime/60)

            prod_padding_chance = [i[1] for i in user_ratings if i[0] == current_user][0]

            if (prod_padding_chance <= 2):
                prod_padding = static.DUMMY_PRODUCTIVITY_LOW[randint(0, len(static.DUMMY_PRODUCTIVITY_LOW)-1)]/100

            elif ((prod_padding_chance <= 5) and (prod_padding_chance > 2)):
                prod_padding = static.DUMMY_PRODUCTIVITY_AVG[randint(0, len(static.DUMMY_PRODUCTIVITY_AVG)-1)]/100

            else:
                prod_padding = static.DUMMY_PRODUCTIVITY_HIGH[randint(0, len(static.DUMMY_PRODUCTIVITY_HIGH)-1)]/100
            
            padded_arc_shift = arc_shift * prod_padding

            shift_padding = (shift_length - padded_arc_shift)

            split_padding = (shift_padding)/2

            m = split_padding * 60

            dt_start = dt_start + datetime.timedelta(minutes=m)

            dt_end = dt_end - datetime.timedelta(minutes=m)

            # log in
            log_record = add2log(current_user, 100, current_job, current_machine, dt_start)
            data.append(log_record)

            #log out
            log_record = add2log(current_user, 101, current_job, current_machine, dt_end)
            data.append(log_record)

            dt_length = dt_end - dt_start # calculate real shift length

            #print("end: " + str(dt_end) + " start:" + str(dt_start) + " = " + str(dt_length))
            (h, m, s) = str(dt_length).split(':')

            decimal_time = int(h) + (int(m) / 60) + (int(s) / 3600) # convert datetime to decimal shift length

            inc_job_hrs(current_job, decimal_time, current_day) #increment worked hours into jobs.csv

            ## Chance to add an Unknown record
            if randint(0, 1000) == 1:
                log_record = add2log(current_user, 404, randint(0, 99999), randint(1, 500), dt_start)
                data.append(log_record)


# task_id, uid, type, job_id, machine_id, datetime
write2csv('log.csv', data, ['task_id', 'user', 'type', 'job', 'machine', 'datetime'])
